In [1]:
#==============================================================================================
# sbert 모델을 각 4가지 평가 데이터셋으로 한꺼번에 평가하는 예시임
# => 평가 데이터 셋 : Korsts, Kluests, gluests, stsb_multi_mt 
# => 각 개별적 eval 출력 하는 예시는 sbert-test2.ipynb 참조
#==============================================================================================
import os
import sys
import time
from sentence_transformers import models, losses
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.evaluation import SimilarityFunction

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert-test", logfilename="../../log/s-bert-test")
device = GPU_info()
seed_everything(111)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


logfilepath:../../log/s-bert-test_2022-11-07.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
## 평가 설정 parameter 들 ########################################

# 평가할 bert 모델 혹은 sbert모델 경로
bisSKKobertModel = 0  # skt/kobert-base-V1 허깅페이스 모델 사용시에는 1로 해줌
model_path = "../../data11/model/sbert/scratch/mdistilbert-scratchV-8-sts/"
#model_path = "kpfbert-sts"


# 유사도 측정방식(COSINE, EUCLIDEAN, MANHATTAN, DOT_PRODUCT 중 선택 , 모두 spearman 방식임)
# => None 이면 아래 값들중 MAX 값 추력함
#main_similarity = None
main_similarity = SimilarityFunction.COSINE
#main_similarity = SimilarityFunction.EUCLIDEAN
#main_similarity = SimilarityFunction.MANHATTAN
#main_similarity = SimilarityFunction.DOT_PRODUCT

# 평가시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
output_path = 'eval'
os.makedirs(output_path, exist_ok=True)
csvfilename = 'scratch'   # 유사도 저장할 파일명 (eval/similarity_evaluation_{csffilename}.csv 형식으로 저장)

# 평가 sts 형태의 말뭉치 파일들
use_korsts = 1     #  korsts 파일
use_kluests = 1    # kluests_v1.1 파일
use_glue_sts = 1   # true이면 영문 glue_sts 데이터셋 추가하여 평기 시킴
use_stsb = 1     # true이면 영문 stsb_multi_mt데이터셋 추가하여 평가시킴.)

# * 모델에 상관없이 영어는 소문자로 입력하겠다는 뜻(glue는 소문자로만 비교하는게 더 적확함), 한국어는 True도 상관없음
do_lower_case = False
    
# korsts tsv 파일 경로
korsts_file = '../../data11/korpora/korsts/tune_test.tsv'

# kluests json 파일 경로
kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

# glue test tsv 파일경로 => validataion 데이터셋으로 eval 구하므로 주석처리함
#glue_test_file = '../../data11/korpora/gluestsb/gluestsb-test.tsv'

eval_batch_size = 64 # 배치파일사이즈 = 크면 eval 시간이 단축됨.단 gpu 메모리 올라감.
max_seq_length = 72

In [3]:
# 모델과 tokenizer 를 불러옴
# => **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=do_lower_case)

#========================================================================================================
# skt/kobert 모델은 tokenizer을 XLNET Tokenizer 이므로, 자체 KoBERTTOkenizer 를 불러와서 사용해야 함.
# => 설치 : !pip install 'git+https://github.com/SKTBrain/KOBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# => 출처 : https://velog.io/@m0oon0/KoBERT-%EC%82%AC%EC%9A%A9%EB%B2%95
if bisSKKobertModel == 1:
    from kobert_tokenizer import KoBERTTokenizer
    word_embedding_model.tokenizer = KoBERTTokenizer.from_pretrained(model_path)
#========================================================================================================

# embedding 길이를 재조정 필요할때 auto_model.resize_token_embeddings 해줌
print(f'token_len:{len(word_embedding_model.tokenizer)}')
#word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
print(model)

#model = SentenceTransformer(smodel_path, device='cpu')
#model = SentenceTransformer(smodel_path, cache_folder=output_path)
#model.to(device)
logger.info(f"*main_similarity: {main_similarity}")
logger.info(f"*model path: {model_path}")
logger.info(f'{model}')

2022-11-07 09:17:44,666 - s-bert-test - INFO - *main_similarity: SimilarityFunction.COSINE
2022-11-07 09:17:44,696 - s-bert-test - INFO - *model path: ../../data11/model/sbert/scratch/mdistilbert-scratchV-8-sts/
2022-11-07 09:17:44,698 - s-bert-test - INFO - SentenceTransformer(
  (0): Transformer({'max_seq_length': 72, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


token_len:30022
SentenceTransformer(
  (0): Transformer({'max_seq_length': 72, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [4]:
from typing import Dict, List, Optional

# 평가 함수 정의
def evalsts(model,                      # 모델
            output_path,                # eval 파일 출력 dir         
            eval_datasets: List[int],   # eval 데이터셋(리스트)
            main_similarity,            # 평가방식(cosine....)
            batch_size,                 # batch_size
            csvfilename,                # eval 출력 파일 명 
           ):

    start = time.time()
    
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(eval_datasets, main_similarity=main_similarity, batch_size=eval_batch_size, name=csvfilename, show_progress_bar=True)
    result = test_evaluator(model, output_path=output_path)
    
    logger.info(f"\n")
    logger.info(f'=== result: {result} ===')
    logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
    logger.info("=====================================================")
    logger.info("\n")
    

In [5]:
# Korsts 평가 시작 
from datasets import load_dataset

if use_korsts == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*korsts file({korsts_file})")
    
    with open(korsts_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for line in lines:
            s1, s2, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts=[s1,s2], label=score))
            
    # 평가 시작
    evalsts(model, output_path, test_samples, main_similarity, eval_batch_size, csvfilename)
    

2022-11-07 09:17:44,820 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:44,821 - s-bert-test - INFO - *korsts file(../../data11/korpora/korsts/tune_test.tsv)


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

2022-11-07 09:17:50,526 - s-bert-test - INFO - 

2022-11-07 09:17:50,527 - s-bert-test - INFO - === result: 0.022374401970361908 ===
2022-11-07 09:17:50,528 - s-bert-test - INFO - === 처리시간: 5.703 초 ===
2022-11-07 09:17:50,529 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:50,529 - s-bert-test - INFO - 



In [6]:
# Kluests 평가 시작 
from datasets import load_dataset
import json
    
if use_kluests == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*Kluests file({kluests_file})")
    
    with open(kluests_file, "r", encoding='utf-8') as fIn2:
        data = json.load(fIn2)
        for el in data:
            s1 = el["sentence1"]
            s2 = el["sentence2"]
            score = el["labels"]['label']
            test_samples.append(InputExample(texts=[s1,s2], label=score))
            
    # 평가 시작
    evalsts(model, output_path, test_samples, main_similarity, eval_batch_size, csvfilename)

2022-11-07 09:17:50,538 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:50,539 - s-bert-test - INFO - *Kluests file(../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json)


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

2022-11-07 09:17:50,803 - s-bert-test - INFO - 

2022-11-07 09:17:50,804 - s-bert-test - INFO - === result: -0.005887557577689524 ===
2022-11-07 09:17:50,804 - s-bert-test - INFO - === 처리시간: 0.258 초 ===
2022-11-07 09:17:50,805 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:50,805 - s-bert-test - INFO - 



In [7]:
# gluests 평가 시작 
from datasets import load_dataset
    
if use_glue_sts == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*gluests")
    
    #glue stsb valildation 데이터셋 불러옴(subset : "stsb" = 1,500)
    glue_stsb_dataset = load_dataset("glue","stsb", split="validation")
    for data in glue_stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    '''
    # glue-test는 허깅페이스 데이터셋에는 labe(score)가 -1로 등록되어 있어서, 실제 glue 사이트에서 org파일을 받아서 불러옴
    # => 해당 sts-test.tsv 파일에서 몇가지 주석달린 문장들은 제거함(약 300개)
    with open(glue_test_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for idx, line in enumerate(lines):
            if idx == 0:
                continue
            _, _, _, _, score, text_a, text_b = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    '''
            
    # 평가 시작
    evalsts(model, output_path, test_samples, main_similarity, eval_batch_size, csvfilename)

2022-11-07 09:17:50,814 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:50,815 - s-bert-test - INFO - *gluests
Reusing dataset glue (/MOCOMSYS/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

2022-11-07 09:17:52,599 - s-bert-test - INFO - 

2022-11-07 09:17:52,600 - s-bert-test - INFO - === result: 0.09536265833568616 ===
2022-11-07 09:17:52,601 - s-bert-test - INFO - === 처리시간: 0.641 초 ===
2022-11-07 09:17:52,601 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:52,602 - s-bert-test - INFO - 



In [8]:
# stsb_multi_mt 평가 시작 
from datasets import load_dataset
    
if use_stsb == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*stsb_multi_mt")
    
    stsb_dataset = load_dataset("stsb_multi_mt", name="en", split="test")
    for data in stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["similarity_score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
            
    # 평가 시작
    evalsts(model, output_path, test_samples, main_similarity, eval_batch_size, csvfilename)

2022-11-07 09:17:52,609 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:52,611 - s-bert-test - INFO - *stsb_multi_mt
Reusing dataset stsb_multi_mt (/MOCOMSYS/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

2022-11-07 09:17:54,311 - s-bert-test - INFO - 

2022-11-07 09:17:54,312 - s-bert-test - INFO - === result: -0.015482966791897104 ===
2022-11-07 09:17:54,312 - s-bert-test - INFO - === 처리시간: 0.565 초 ===
2022-11-07 09:17:54,313 - s-bert-test - INFO - =====================================================
2022-11-07 09:17:54,314 - s-bert-test - INFO - 

